# --- Day 10: Hoof It ---

You all arrive at a Lava Production Facility on a floating island in the sky. As the others begin to search the massive industrial complex, you feel a small nose boop your leg and look down to discover a reindeer wearing a hard hat.

The reindeer is holding a book titled *"Lava Island Hiking Guide"*. However, when you open the book, you discover that most of it seems to have been scorched by lava! As you're about to ask how you can help, the reindeer brings you a blank topographic map of the surrounding area (your puzzle input) and looks up at you excitedly.

Perhaps you can help fill in the missing hiking trails?

---

The topographic map indicates the height at each position using a scale from `0` (lowest) to `9` (highest). For example:

```
0123
1234
8765
9876
```

Based on un-scorched scraps of the book, you determine that a good hiking trail is as long as possible and has an even, gradual, uphill slope. For all practical purposes, this means that a hiking trail is any path that:

- Starts at height `0`,
- Ends at height `9`, and
- Always increases by a height of exactly `1` at each step.

Hiking trails never include diagonal steps — only up, down, left, or right (from the perspective of the map).

You look up from the map and notice that the reindeer has helpfully begun to construct a small pile of pencils, markers, rulers, compasses, stickers, and other equipment you might need to update the map with hiking trails.

---

### Definitions

A **trailhead** is any position that starts one or more hiking trails. In this puzzle, these positions will always have height `0`. 

You determine that a trailhead's **score** is the number of `9`-height positions reachable from that trailhead via a hiking trail. 

---

### Examples

#### Example 1

For the map:

```
0123
1234
8765
9876
```

The single trailhead in the top left corner has a score of `1` because it can reach a single `9` (the one in the bottom left).

---

#### Example 2

This trailhead has a score of `2`:

```
...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9
```

(The positions marked `.` are impassable tiles to simplify these examples; they do not appear on your actual topographic map.)

---

#### Example 3

This trailhead has a score of `4` because every `9` is reachable via a hiking trail except the one immediately to the left of the trailhead:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

---

#### Example 4

This topographic map contains two trailheads. The trailhead at the top has a score of `1`, while the trailhead at the bottom has a score of `2`:

```
10..9..
2...8..
3...7..
4567654
...8..3
...9..2
.....01
```

---

#### Example 5

Here's a larger example:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

This larger example has `9` trailheads. Considering the trailheads in reading order, they have scores of `5`, `6`, `5`, `3`, `1`, `3`, `5`, `3`, and `5`. Adding these scores together, the sum of the scores of all trailheads is `36`.

---

The reindeer gleefully carries over a protractor and adds it to the pile. 

**What is the sum of the scores of all trailheads on your topographic map?**

In [1]:
from dataclasses import dataclass, field
import numpy as np


@dataclass
class Map:
    grid: np.ndarray

    def __getitem__(self, idx):
        return self.grid[idx]

    def get_neighbourhood(self, pos: tuple, radius: int = 1):
        slices = tuple(
            slice(max(p - radius, 0), min(p + radius + 1, dim))
            for p, dim in zip(pos, self.grid.shape)
        )
        return self.grid[slices]

    @classmethod
    def from_file(cls, filepath: str) -> "Map":
        with open(filepath, "r", encoding="utf-8") as f:
            grid = np.array([list(line.strip()) for line in f])
        grid = np.pad(
            grid.astype(int), ((1, 1), (1, 1)), "constant", constant_values=-1
        )
        return cls(grid)


@dataclass
class Hiker:
    topology: Map
    trailheads: np.ndarray = field(init=False)
    trails_map: np.ndarray = field(init=False)
    good_trails: int = 0
    steps_mask: np.ndarray = field(
        default_factory=lambda: np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
    )

    def __post_init__(self):
        self.trailheads = np.argwhere(self.topology.grid == 0)
        self.trails_map = np.ones_like(self.topology.grid) * -1

    def wander(self, only_unexplored: bool = True) -> "Hiker":
        for pos in self.trailheads:
            self.trails_map = np.ones_like(self.topology.grid) * -1
            self.hike_trail(pos, only_unexplored)
        return self

    def hike_trail(self, pos: tuple, only_unexplored: bool = True):
        self.update_trails_map(pos)
        if self.reached_summit(pos):
            self.good_trails += 1
            return

        slopes = self.even_uphill_slopes(pos)
        if only_unexplored:
            slopes = [s for s in slopes if self.trails_map[s] == -1]

        for slope in slopes:
            self.hike_trail(slope, only_unexplored)

    def reached_summit(self, pos: tuple):
        return self.topology[*pos] == 9

    def even_uphill_slopes(self, pos: tuple):
        roi = self.topology.get_neighbourhood(pos)
        step_gradient = (roi - self.topology[*pos]) * self.steps_mask
        return [
            (
                pos[0] + step[0] - self.steps_mask.shape[0] // 2,
                pos[1] + step[1] - self.steps_mask.shape[1] // 2,
            )
            for step in np.argwhere(step_gradient == 1)
        ]

    def update_trails_map(self, pos):
        self.trails_map[*pos] = self.topology[*pos]


hiker = Hiker(topology=Map.from_file("./example.txt"))
hiker.wander(only_unexplored=True).good_trails

36

In [2]:
hiker = Hiker(topology=Map.from_file("./input.txt"))
hiker.wander(only_unexplored=True).good_trails

538

# --- Part Two ---

The reindeer spends a few minutes reviewing your hiking trail map before realizing something, disappearing for a few minutes, and finally returning with yet another slightly-charred piece of paper.

The paper describes a second way to measure a trailhead called its **rating**. A trailhead's rating is the **number of distinct hiking trails** which begin at that trailhead. 

---

### Examples

#### Example 1
For the following map:

```
.....0.
..4321.
..5..2.
..6543.
..7..4.
..8765.
..9....
```

The map has a single trailhead; its rating is `3` because there are exactly three distinct hiking trails which begin at that position:

1. Trail 1:
```
.....0.   
..4321.   
..5....   
..6....   
..7....   
..8....   
..9....   
```

2. Trail 2:
```
.....0.   
.....1.   
.....2.   
..6543.   
..7....   
..8....   
..9....   
```

3. Trail 3:
```
.....0.   
.....1.   
.....2.   
.....3.   
.....4.   
..8765.   
..9....   
```

---

#### Example 2
For the following map:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This map contains a single trailhead with a **rating of 13**.

---

#### Example 3
For the following map:

```
012345
123456
234567
345678
4.6789
56789.
```

This map contains a single trailhead with a **rating of 227**, because there are `121` distinct hiking trails that lead to the `9` on the right edge and `106` distinct hiking trails that lead to the `9` on the bottom edge.

---

#### Larger Example

Using the larger example from before:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

Considering its trailheads in reading order, they have ratings of:

- `20`, `24`, `10`, `4`, `1`, `4`, `5`, `8`, and `5`.

The sum of all trailhead ratings in this larger example topographic map is `81`.

---

The reindeer gleefully crafts tiny flags out of toothpicks and bits of paper, using them to mark trailheads on your topographic map. 

**What is the sum of the ratings of all trailheads?**

In [3]:
hiker = Hiker(topology=Map.from_file("./example.txt"))
hiker.wander(only_unexplored=False).good_trails

81

In [4]:
hiker = Hiker(topology=Map.from_file("./input.txt"))
hiker.wander(only_unexplored=False).good_trails

1110